In [1]:
### pakiety - potrzebne w colabie
!pip install datasets
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer
!pip install transformers[torch]
!pip install accelerate==0.30.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [2]:
### login do huggingface - potrzebny w colabie
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "pl", split="train")
common_voice_test = load_dataset("mozilla-foundation/common_voice_17_0", "pl", split="test")

# ograniczenie wielkości datasetów do testów
#common_voice_train = common_voice_train.select(range(1024))
common_voice_test = common_voice_test.select(range(1024))

In [5]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence,variant
0,"Uważam też, że grupy polityczne muszą dziś zareagować, przedstawiając wspólne stanowisko w tej sprawie",
1,W kontekście obecnego kryzysu gospodarczego ta kwestia wiąże się ściśle z zatrudnieniem,
2,"""Podpierał się o wejście ciemnego sklepienia, i kleszczami obcinał paznokcie u rąk.""",
3,"""Nie otworzę okna!""",
4,W tym przypadku odpowiedź na to pytanie jest prosta,
5,"Wierzę, że ma pan zarówno wizję, jak i odwagę",
6,"Chodzi tu o utrzymanie państwowych monopoli, a my wiemy, do czego to prowadzi",
7,"""Był bardziej skurczony i miał czerwieńsze oczy, niż zwykle.""",
8,"Przyłączam się do tych, którzy powiedzieli, że była to doskonała debata",
9,"My, mieszkańcy Europy Środkowej, w tym Węgrzy, byliśmy i nieodmiennie pozostajemy zainteresowani zjednoczeniem Europy",


In [7]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\«\»\–\—\’\„\…]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [8]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

show_random_elements(common_voice_train.remove_columns(["path","audio"]))

Map:   0%|          | 0/20729 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

,sentence,variant
0,pragnienia wolności żadnego narodu nie da się zdławić na zawsze,
1,prześmiewcze hasło bób hummus włoszczyzna,
2,warto też rozważyć przyczyny tych niebezpiecznych zjawisk,
3,wymiar sprawiedliwości i administracja publiczna są reformowane a walka z korupcją przybiera na sile,
4,i był już teraz spokojny,
5,jest członkiem ponieważ uczyniła wszystko naprawdę wszystko aby nim zostać,
6,reorientacja wsparcia w ramach europejskiego funduszu rybołówstwa,
7,zapewniona będzie ponadto wspólna forma szkolenia,
8,powiedzmy im,
9,dodałbym tutaj również że kwestia przemytu jest dramatycznym problemem tego kraju,


In [9]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[ą]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[ę]', 'e', batch["sentence"])
    batch["sentence"] = re.sub('[ó]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[ł]', 'l', batch["sentence"])
    batch["sentence"] = re.sub('[ż]', 'rz', batch["sentence"])
    batch["sentence"] = re.sub('[ń]', 'ni', batch["sentence"])
    batch["sentence"] = re.sub('[ć]', 'ci', batch["sentence"])
    batch["sentence"] = re.sub('[ś]', 'si', batch["sentence"])
    batch["sentence"] = re.sub('[ź]', 'zi', batch["sentence"])
    batch["sentence"] = re.sub('[ü]', 'u', batch["sentence"])
    return batch

common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)

Map:   0%|          | 0/20729 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

In [10]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/20729 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

In [11]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [12]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [13]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [14]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [15]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [16]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/6aa470cdebcd2b54f9346df4f609e8a6712f266e096cb7f286d7b80c6fe1f12b/pl_train_0/common_voice_pl_21655246.mp3'

In [17]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/6aa470cdebcd2b54f9346df4f609e8a6712f266e096cb7f286d7b80c6fe1f12b/pl_train_0/common_voice_pl_21655246.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.05379274e-06, -3.55337511e-06, -4.30422733e-06]),
 'sampling_rate': 48000}

In [18]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [19]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/6aa470cdebcd2b54f9346df4f609e8a6712f266e096cb7f286d7b80c6fe1f12b/pl_train_0/common_voice_pl_21655246.mp3',
 'array': array([-2.96461532e-21,  3.17637355e-21, -2.11758237e-20, ...,
         7.11472239e-07, -3.45709395e-07, -2.75473758e-06]),
 'sampling_rate': 16000}

In [20]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

jednak spoleczenistwo oparte na solidarnosici nie morze w ten sposub postrzegaci procesu starzenia sie


In [21]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: w rezolucjach takich jak ta zawsze urzywa sie niezwykle dyplomatycznego jezyka
Input array shape: (110016,)
Sampling rate: 16000


In [22]:
from transformers import Wav2Vec2FeatureExtractor

model = "facebook/hubert-large-ls960-ft"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model)

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [23]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [24]:
from transformers import HubertForCTC

model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-

In [25]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [26]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/20729 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

In [27]:
print(common_voice_train.column_names)

['input_values', 'input_length', 'labels']


In [28]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [29]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [30]:
wer_metric = load_metric("wer")

<ipython-input-30-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


The repository for wer contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wer.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [31]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [32]:
def change_grad(module, on):
    for param in module.parameters():
        param.requires_grad=on
    module.requires_grad=on

In [33]:
change_grad(model, True)

In [34]:
model.freeze_feature_encoder()

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./drive/MyDrive/hubert/out",
  group_by_length=False,
  per_device_train_batch_size=32,
  gradient_accumulation_steps=1,
  eval_accumulation_steps=4,
  eval_strategy="steps",
  num_train_epochs=23, # powinno być około 30
  gradient_checkpointing=True,
  fp16=True,
  save_steps=1,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=3,
)

In [64]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [62]:
trainer.train()

KeyboardInterrupt: 